In [142]:
import math
import numpy as np

In [143]:
np.random.seed(150000)

In [144]:
def generate_asset_price(S0, r, sigma, dt, rand):
    '''
    $S_{t} = S_{t-\Delta t}e^{(r-(1/2)\sigma^{2})\Delta t+\sigma\sqrt{\Delta t}z_{t}}$

    t = ${\Delta t, 2\Delta t, ... T}$

    $z_{t}$ = standard normally distributed random variables
    '''
    S = S0 * np.exp(np.cumsum((r - 0.5 * sigma ** 2) * dt \
                                 + sigma * math.sqrt(dt) * np.random.standard_normal((M + 1, I)), 
                                 axis=0))
    S[0] = S0
    
    return S

def put_payoff(S, K):
    return np.maximum(K - S, 0)

def call_payoff(S, K):
    return np.maximum(S - K, 0)

In [145]:
#### Model Parameters ####
S0 = 36.  # initial stock level
K = 40.  # strike price
T = 1.0  # time-to-maturity
r = 0.06  # short rate
sigma = 0.2  # volatility

### Simulation Parameters ###
I = 25000 # number of simulations
M = 50 # number of time steps
dt = T / M # time step
df = math.exp(-r * dt) # discount factor

In [164]:
rand_arr = np.random.standard_normal((M, I)) # standard normal distribution
S = generate_asset_price(S0, r, sigma, dt, rand_arr[t]) # (M,I) matrix
payoff = put_payoff(S, K) # (M,I) matrix
V = payoff[-1] # Present Value Vector (Initialization)

In [165]:
### American Option Valuation by Backwards Induction ###
for t in range(M - 1, 0, -1):
    # fit stock prices to call option values discounted one period (at time t)
    # V is continously updated as you work backwards
    reg_func = np.polyfit(x=S[t], y=V * df, deg=3)
    
    # evaluate the regression function to get the y-values (option values)
    C = np.polyval(reg, S[t])
    
    # where the calculated payoff of max(K-S, 0) is greater than approximated value, exercise the option
    # V is continously updated as you work backwards
    V = np.where(payoff[t] > C, payoff[t], V * df)

V0 = df * np.sum(V) / I  # LSM estimator - average of discounted values

print ("American put option value %5.3f" % V0)

American put option value 4.478
